In [27]:
import numpy as np
import pandas as pd

In [28]:
def gaussian(x, mu, sigma):
    return 1 / (np.sqrt(2*np.pi)*sigma) \
    * np.exp(-np.power(x-mu,2)/(2*np.power(sigma,2)))

In [190]:
df = pd.read_csv('data/lat_of_fish.csv')
df.head()

,sst,lat_of_mackerel,lat_of_menhaden,time,seasonal
0,6.399033,50.000000,53.000000,2018.000000,-1.382851
1,6.401424,50.003253,53.003253,2018.083472,-1.245809
2,6.404461,50.007385,53.007385,2018.166945,-0.994595
3,6.407805,50.011934,53.011934,2018.250417,-0.704210
4,6.411156,50.016494,53.016494,2018.333890,-0.125891


In [192]:
def fish_density(lat, lat_of_mackerel, lat_of_menhaden):
    return gaussian(lat, lat_of_mackerel, np.sqrt(6))\
           + gaussian(lat, lat_of_menhaden, np.sqrt(6))
array = fish_density(np.linspace(40, 60), 50, 53)

In [193]:
def best_reward(start_lat, move_time, lat_of_mackerel, lat_of_menhaden):
    if start_lat <= np.mean([lat_of_mackerel, lat_of_menhaden]):
        return 1/week * (8*week-move_time) * fish_density(start_lat+0.2*move_time, lat_of_mackerel, lat_of_menhaden)
    if start_lat > np.mean([lat_of_mackerel, lat_of_menhaden]):
        return 1/week * (8*week-move_time) * fish_density(start_lat+0.2*move_time, lat_of_mackerel, lat_of_menhaden)

In [194]:
def best_reward_total(start_lat, move_time, sample=50):
    sum = 0
    df_sample = df.sample(sample, random_state=0)
    for index, row in df_sample.iterrows():
        sum += best_reward(start_lat, move_time, row['lat_of_mackerel'], row['lat_of_menhaden'])
    sum /= sample
    sum *= len(df)
    return sum

In [82]:
from sko.SA import SA
from plotnine import *

In [213]:
def best_reward_for_sa(x):
    if x[1] < 0 or x[1] > 36 or x[0] < 50 or x[0] > 62:
        return np.Inf
    # x[0] 表示纬度, x[1] 表示移动时间
    return -best_reward_total(x[0], x[1], sample=10)

week = 1
x_history_list, y_history_list = [], []
x_result_list, y_result_list = [], []
for i in range(6):
    sa = SA(func=best_reward_for_sa, x0=[53, 10], T_max=1, T_min=1e-9, L=300, max_stay_counter=150)
    x_result, y_result = sa.run()
    x_history_list.append(sa.best_x_history)
    y_history_list.append(sa.best_y_history)
    x_result_list.append(x_result)
    y_result_list.append(y_result)

In [225]:
np.array(y_history_list[1])

array([1.00000000e+01, 1.18668361e+00, 2.33096051e-01, 2.73893100e-01,
       2.73893100e-01, 2.73893100e-01, 2.73893100e-01, 1.37545030e-01,
       1.37545030e-01, 1.37545030e-01, 1.37545030e-01, 4.35600299e-02,
       4.35600299e-02, 4.35600299e-02, 2.35878793e-02, 2.35878793e-02,
       1.70323250e-02, 1.70323250e-02, 1.62078395e-02, 1.80245776e-03,
       7.65574512e-04, 2.12195501e-04, 2.12195501e-04, 2.12195501e-04,
       1.70479886e-04, 1.70479886e-04, 1.10009202e-04, 7.63459413e-05,
       6.85073073e-06, 4.37542922e-05, 4.37542922e-05, 4.37542922e-05,
       4.37542922e-05, 7.24382758e-06, 8.44238847e-06, 8.44238847e-06,
       6.73513952e-06, 4.07723780e-06, 2.34461258e-06, 1.37606078e-06,
       1.37606078e-06, 8.69547931e-07, 8.69547931e-07, 6.35735751e-07,
       6.35735751e-07, 6.35735751e-07, 1.07301940e-07, 4.96072868e-08,
       5.46164795e-08, 5.46164795e-08, 1.69037367e-08, 1.69037367e-08,
       1.69037367e-08, 1.69037367e-08, 1.69037367e-08, 1.15803927e-08,
      

In [238]:
df_list = []
for i in range(6):
    df_temp = pd.DataFrame({
        'x': np.array(x_history_list[i])[:,0],
        't': np.array(x_history_list[i])[:,1],
        'reward': -np.array(y_history_list[1]),
        'time': i,
    })
    df_list.append(df_temp)
df_temp = pd.concat(df_list)
df_temp = df_temp.drop(labels=0)
df_temp

,x,t,reward,time
1,53.041044,1.223879e+00,1092.855776,0
2,53.041044,1.223879e+00,1155.082418,0
3,54.094154,3.120265e-01,1235.736650,0
4,53.556634,4.514678e-02,1235.736650,0
5,53.556634,4.514678e-02,1235.736650,0
...,...,...,...,...
53,52.511669,5.711937e-10,1283.571802,5
54,52.511669,5.711937e-10,1283.571802,5
55,52.511669,5.711937e-10,1283.571802,5
56,52.511669,5.711937e-10,1283.571802,5


In [240]:
(
    ggplot(df_temp, aes('x', 't', color='reward'))
    + geom_path()
    + geom_point(aes(size='reward'))
    + facet_wrap('time')
    + labs(x='Start Latitude',
           y='Voyage Time',
           title='Iterative Process of Simulated Annealing Algorithm')
).save('img/fish/iterative process of simulated annealing algorithm.png')

c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: img/fish/iterative process of simulated annealing algorithm.png


In [40]:
array = np.concatenate([np.array(sa.best_x_history),
                        np.array(sa.best_y_history).reshape(-1,1)], axis=1)
df_result = pd.DataFrame(array, columns=['start_lat', 'move_time', 'reward'])
df_result['reward'] = -df_result['reward']

(
    ggplot(df_result,aes(x='start_lat', y='move_time'))
    + geom_point(aes(color='reward', size=1.5))
    + geom_path(alpha=.8, color='coral')
    + labs(x='Start Latitude',
           y='Voyage Time',
           title='Relation of Remuneration to Latitude of Origin and Time of Voyage')
).save('img/fish/relation of remuneration to latitude of origin and time of voyage.png')

c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: img/fish/relation of remuneration to latitude of origin and time of voyage.png


In [200]:
def best_reward_for_sa_with_lat_limit(x):
    # x[0] 表示移动时间
    if x[0] < 0 or x[0] > 8 * week:
        return np.Inf
    return -best_reward_total(50, x[0], sample=10)

week = 7
sa2 = SA(func=best_reward_for_sa_with_lat_limit, x0=[8], T_max=1, T_min=1e-9, L=300, max_stay_counter=150)
x_result, y_result = sa2.run()
-y_result * 3179.1

3351873.3854918564

In [42]:
array = np.concatenate([np.array(sa2.best_x_history),
                        np.array(sa2.best_y_history).reshape(-1,1)], axis=1)
df_result = pd.DataFrame(array, columns=['move_time', 'reward'])
df_result['reward'] = -df_result['reward']
df_result = df_result.drop(labels=0)

(
    ggplot(df_result,aes(x='move_time', y='reward'))
    + geom_point(aes(color='reward'), size=1.5)
    + geom_path(alpha=.8, color='coral')
    + labs(x='Voyage Time',
           y='Reward',
           title='Relationship between Earnings and Voyage Time at Latitudes 50°N')
).save('img/fish/relationship between earnings and voyage time at latitudes 50°N.png')

c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: img/fish/relationship between earnings and voyage time at latitudes 50°N.png


In [149]:
def best_reward_for_sa_on_lat_50(x):
    if x[0]<0 or x[0]>8 * week:
        return np.Inf
    return -best_reward(50, x[0], df.loc[index]['lat_of_mackerel'],
                        df.loc[index]['lat_of_menhaden'])

result_dict = {}
for week in [1,3,5,7]:
    print(str(week) + ':')
    x_result_list, y_result_list = [], []
    for index in range(0, len(df), 10):
        print(str(int(index/len(df)*100)) + '%', end=' ')
        if index != 0 and index % 100 == 0:
            print()
        sa3 = SA(func=best_reward_for_sa_on_lat_50,
                 x0=[0])
        x_result, y_result = sa3.run()
        x_result_list.append(x_result)
        y_result_list.append(y_result)
    result_dict[week] = np.concatenate([np.array(x_result_list), np.array(y_result_list).reshape(-1,1)], axis=1)
    print()
result_dict

1:
0% 1% 3% 5% 6% 8% 10% 11% 13% 15% 16% 
18% 20% 21% 23% 25% 26% 28% 30% 31% 33% 
35% 36% 38% 40% 41% 43% 45% 46% 48% 50% 
51% 53% 55% 56% 58% 60% 61% 63% 65% 66% 
68% 70% 71% 73% 75% 76% 78% 80% 81% 83% 
85% 86% 88% 90% 91% 93% 95% 96% 98% 
3:
0% 1% 3% 5% 6% 8% 10% 11% 13% 15% 16% 
18% 20% 21% 23% 25% 26% 28% 30% 31% 33% 
35% 36% 38% 40% 41% 43% 45% 46% 48% 50% 
51% 53% 55% 56% 58% 60% 61% 63% 65% 66% 
68% 70% 71% 73% 75% 76% 78% 80% 81% 83% 
85% 86% 88% 90% 91% 93% 95% 96% 98% 
5:
0% 1% 3% 5% 6% 8% 10% 11% 13% 15% 16% 
18% 20% 21% 23% 25% 26% 28% 30% 31% 33% 
35% 36% 38% 40% 41% 43% 45% 46% 48% 50% 
51% 53% 55% 56% 58% 60% 61% 63% 65% 66% 
68% 70% 71% 73% 75% 76% 78% 80% 81% 83% 
85% 86% 88% 90% 91% 93% 95% 96% 98% 
7:
0% 1% 3% 5% 6% 8% 10% 11% 13% 15% 16% 
18% 20% 21% 23% 25% 26% 28% 30% 31% 33% 
35% 36% 38% 40% 41% 43% 45% 46% 48% 50% 
51% 53% 55% 56% 58% 60% 61% 63% 65% 66% 
68% 70% 71% 73% 75% 76% 78% 80% 81% 83% 
85% 86% 88% 90% 91% 93% 95% 96% 98% 


{1: array([[ 0.        , -1.91840532],
        [ 0.        , -1.90734859],
        [ 0.        , -1.89888193],
        [ 0.        , -1.88995161],
        [ 0.        , -1.880852  ],
        [ 0.        , -1.87165447],
        [ 0.        , -1.86233302],
        [ 0.        , -1.85289249],
        [ 0.        , -1.8433353 ],
        [ 0.        , -1.83366308],
        [ 0.        , -1.82387776],
        [ 0.        , -1.81398127],
        [ 0.        , -1.80397558],
        [ 0.        , -1.79386265],
        [ 0.        , -1.78364449],
        [ 0.        , -1.77332311],
        [ 0.        , -1.76290055],
        [ 0.        , -1.75237887],
        [ 0.        , -1.74176017],
        [ 0.        , -1.73104653],
        [ 0.        , -1.72024008],
        [ 0.        , -1.70934295],
        [ 0.        , -1.69835731],
        [ 0.        , -1.68728532],
        [ 0.        , -1.67612918],
        [ 0.        , -1.66489109],
        [ 0.        , -1.65357326],
        [ 0.        , -1.

In [201]:
df_temp = pd.DataFrame({
    'reward': [2919398.51, 2958061.74, 3173207.71, 3351873.39],
    'day': [1,3,5,7]
})

(
    ggplot(df_temp, aes('day', 'reward', fill='reward'))
    + geom_col()
    + labs(x='Time at Sea, [Day]',
           y='Reward, [Pounds]',
           title='the Relationship between Time at Sea and Income',
           color='Reward')
).save('img/fish/the relationship between time spent at sea and income.png')

c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: img/fish/the relationship between time spent at sea and income.png


In [150]:
df_list = []

for result in result_dict:
    df_temp = pd.DataFrame({
        't': result_dict[result][:,0],
        'reward': -result_dict[result][:,1] * 3179.1 * 12,
        'day': result,
        'time': np.linspace(2018, 2068, len(result_dict[result]))
    })
    df_list.append(df_temp)

df_temp = pd.concat(df_list)
df_temp

,t,reward,day,time
0,0.000000,73185.628347,1,2018.000000
1,0.000000,72763.822766,1,2018.847458
2,0.000000,72440.826528,1,2019.694915
3,0.000000,72100.141864,1,2020.542373
4,0.000000,71752.999175,1,2021.389831
...,...,...,...,...
55,10.065421,63934.071030,7,2064.610169
56,10.191414,63739.464104,7,2065.457627
57,10.317367,63544.916111,7,2066.305085
58,10.443237,63350.427479,7,2067.152542


In [176]:
(
    ggplot(df_temp, aes(color='factor(day)', group='day'))
    + geom_line(aes('time','reward'))
    + geom_hline(aes(yintercept=60700), linetype='--')
    + labs(x='Year',
           y='Reward, [Pounds]',
           title='Relationship between Average Annual Income of Fishermen and Time at Sea',
           color='Time at Sea, [Day]')
).save('img/fish/relationship between average annual income of fishermen and time at sea.png')

c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: img/fish/relationship between average annual income of fishermen and time at sea.png


In [175]:
(
    ggplot(df_temp, aes(color='factor(day)', group='day'))
    + geom_line(aes('time','t'))
    + labs(x='Year',
           y='Voyage Time',
           title='the Relationship between Time of Voyage and Time at Sea',
           color='Time at Sea, [Day]')
).save('img/fish/the relationship between time of voyage and time at sea.png')


c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: img/fish/the relationship between time of voyage and time at sea.png
